# Generative Adversarial Networks

Применение adversarial loss (более общей идеи, лежащей в основе GANов) позволило решить задачи, которые казались невозможными:

* [Машинный перевод без параллельных данных](https://arxiv.org/pdf/1710.11041.pdf)
* [Циклоганы: перевод изображений в другой домен](https://arxiv.org/abs/1703.10593)
* Колоризация и [Super Resolution](https://arxiv.org/abs/1807.02758)
* [Генерация и морфинг произвольных данных](https://arxiv.org/pdf/1809.11096.pdf) ([тут](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb#scrollTo=HuCO9tv3IKT2) можно поиграться с генерацией бургеров)
* Применения в борьбе с adversarial атаками

Вот постоянно пополняющийся список приложений GANов: https://github.com/nashory/gans-awesome-applications

Сама [статья](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) Яна Гудфеллоу про GANы вышла в конце 2014 года и была процитирована 7687 раз за 4 года.


<img width='500px' src='https://cdn-images-1.medium.com/max/800/1*eWURQXT41pwHvDg1xDiEmw.png'>

Теперь немного формальных определений:

* Пусть $z$ — это вектор из латентного пространства, насэмпленный из нормального распределения.
* $G(z)$ обозначает функцию генератора, которая отображает латентный вектор в пространство данных. Цель $G$ — оценить истинное распределение данных $p_d$, чтобы сэмплировать данные из оцененного распределения $p_g$.
* $D(G(z))$ это вероятность (число от 0 до 1), что выход генератора $G$ является реальным изображением.

$D$ и $G$ играют в минимаксную игру, в которой $D$ старается максимизировать вероятность, что он правильно классифицирует реальные и сгенерированные сэмплы, а $G$ старается минимизировать эту вероятность:

$$\underset{G}{\text{min}} \underset{D}{\text{max}}V(D,G) = \mathbb{E}_{x\sim p_{data}(x)}\big[logD(x)\big] + \mathbb{E}_{z\sim p_{z}(z)}\big[log(1-D(G(x)))\big]$$

[Выясняется](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf), что решение в этой минимаксной игре достигается при $p_g = p_d$ (и дискриминатор в этом случае может угадывать случайно). В реальности модели не всегда могут сойтись к этой точке.

[DCGAN](https://arxiv.org/pdf/1511.06434.pdf) (Deep Convolutional GAN) называют GAN, который явно использует свёртки и транспонированные свёртки в дискриминаторе и генераторе соответственно. Откройте статью -- мы будем идти очень близко с авторами.

## Датасет
Всем надоели цифры, поэтому обучаться мы будем на датасете CelebA ([Large-scale CelebFaces Attributes](Large-scale CelebFaces Attributes)). В датасете на каждую фотку есть её аттрибуты, но мы их пока использовать не будем.

<img width='500px' src='http://mmlab.ie.cuhk.edu.hk/projects/celeba/overview.png'>

Автор, когда готовил эту тетрадку, долго думал, как загрузить датасет, чтобы всем было удобно. Это оказалось трудно, потому что прямых ссылок на него нигде нет, и, соответственно, просто сделать `!wget ...` нельзя. По удачному стечению обстоятельств, неделю назад кто-то [добавил](https://github.com/pytorch/vision/blob/master/torchvision/datasets/celeba.py) скрипты для загрузки этого датасета в сам `torchvision`, но в `pip` новая версия за такой срок ещё не успела появиться, поэтому мы обновимся напрямую из репозитория на гитхабе:

In [0]:
!pip install git+https://github.com/pytorch/vision.git

  Cloning https://github.com/pytorch/vision.git to /tmp/pip-req-build-d78bb8nn
  Stored in directory: /tmp/pip-ephem-wheel-cache-ohnc5_d1/wheels/04/6d/bf/cc14a58bae32d07d1c7d23833dc5ea655e477ff25061b8cd57
Successfully built torchvision
fastai 1.0.48 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: torchvision 0.2.2.post3
    Uninstalling torchvision-0.2.2.post3:
      Successfully uninstalled torchvision-0.2.2.post3


In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [2]:
device = torch.device('cuda:0')  # не забудьте включить GPU

image_size = 64
batch_size = 64

In [3]:
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # Normalize здесь приводит значения в промежуток [-1, 1]
])

dataset = datasets.CelebA('data', download=True, transform=transform)

loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

AttributeError: module 'torchvision.datasets' has no attribute 'CelebA'

In [ ]:
dataset[5][0]

In [ ]:
# посмотрите на данные (вы писали нужный код в колоризации)
# ...

## Модель

Генератор $G$ преобразует латентный вектор $z$ в пространство данных (в нашем случае -- картинки 3x64x64). В статье используют последовательность блоков из транспонированных свёрток, BatchNorm-ов и ReLU. На выходе каждое значение лежит в [-1, 1] (мы делаем TanH), в соответствии с нормализацией, которую мы сделали раньше.

<img width='600px' src='https://pytorch.org/tutorials/_images/dcgan_generator.png'>

In [4]:
device = torch.device('cpu')

In [12]:
num_channels = 3
latent_size = 100
base_size = 64

G = nn.Sequential(
    # input is Z, going into a convolution
    nn.ConvTranspose2d(latent_size, base_size * 8, 4, 1, 0, bias=False),
    nn.BatchNorm2d(base_size * 8),
    nn.ReLU(True),
    
    # (base_size*8) x 4 x 4
    nn.ConvTranspose2d(base_size * 8, base_size * 4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 4),
    nn.ReLU(True),
    
    # (base_size*4) x 8 x 8
    nn.ConvTranspose2d(base_size * 4, base_size * 2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 2),
    nn.ReLU(True),
    
    # (base_size*2) x 16 x 16
    nn.ConvTranspose2d(base_size * 2, base_size, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size),
    nn.ReLU(True),
    
    # (base_size) x 32 x 32
    nn.ConvTranspose2d(base_size, num_channels, 4, 2, 1, bias=False),
    nn.Tanh()
    # (num_channels) x 64 x 64
).to(device)

In [13]:
z = torch.randn(1, latent_size, 1, 1)
G(z)

tensor([[[[ 0.0430, -0.1921, -0.0838,  ..., -0.0961,  0.0013, -0.0193],
          [-0.0952, -0.2367, -0.3027,  ...,  0.0537, -0.0057, -0.1617],
          [ 0.0071, -0.1059, -0.1645,  ..., -0.0812, -0.2847, -0.1735],
          ...,
          [-0.0262, -0.0089, -0.1760,  ..., -0.1663, -0.2924, -0.1015],
          [ 0.0520, -0.0849,  0.0632,  ..., -0.0493, -0.0067, -0.2049],
          [ 0.0066, -0.0965, -0.2745,  ..., -0.0040, -0.1068, -0.1311]],

         [[-0.2534,  0.1518, -0.3848,  ...,  0.0874, -0.3271, -0.0220],
          [-0.0451,  0.1393,  0.2194,  ..., -0.0423, -0.0577,  0.0611],
          [-0.1836,  0.0687, -0.2834,  ...,  0.1847, -0.4244,  0.0374],
          ...,
          [-0.0741, -0.0388,  0.1716,  ..., -0.1594,  0.0327, -0.0295],
          [-0.2991,  0.2001, -0.4033,  ...,  0.0508, -0.3711, -0.0573],
          [-0.0035,  0.1809,  0.0290,  ...,  0.0626,  0.0673,  0.0036]],

         [[ 0.0896, -0.0668, -0.0163,  ..., -0.0538, -0.0037, -0.0130],
          [ 0.1059,  0.0368,  

Дискриминатор -- это обычный бинарный классификатор. В статье он устроен симметрично генератору: Conv2d, BatchNorm, ReLU, Conv2d...

In [16]:
D = nn.Sequential(

    nn.Conv2d(num_channels, base_size , 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size),
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(base_size, base_size * 2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 2),
    nn.LeakyReLU(0.2, inplace=True),
    # state size. (ndf*2) x 16 x 16
    nn.Conv2d(base_size * 2, base_size * 4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 4),
    nn.LeakyReLU(0.2, inplace=True),
    # state size. (ndf*4) x 8 x 8
    nn.Conv2d(base_size * 4, base_size * 8, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 8),
    nn.LeakyReLU(0.2, inplace=True),
    # state size. (ndf*8) x 4 x 4
    nn.Conv2d(base_size * 8, 1, 4, 1, 0, bias=False),
    nn.Sigmoid()
).to(device)

In [17]:
z = torch.randn(1, latent_size, 1, 1).to(device)
D(G(z))

tensor([[[[0.4384]]]], grad_fn=<SigmoidBackward>)

В статье акцентируют внимание на необходимость нестандартной инициализации весов.

In [18]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


# apply рекурсивно применяет применяет функцию ко всем своим подмодулям
G.apply(weights_init)
D.apply(weights_init)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.2, inplace)
  (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.2, inplace)
  (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): LeakyReLU(negative_slope=0.2, inplace)
  (9): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): LeakyReLU(negative_slope=0.2, inplace)
  (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (13): Sigmoid()
)

## Обучение

У GANов, помимо сходимости, есть проблема, что их непонятно, как сравнивать между собой, потому что у нас не один лосс, а два. Поэтому полезнее во время обучения смотреть на генерируемые картинки, а не цифры.

In [ ]:
fixed_noise = torch.randn(opt.batchSize, nz, 1, 1, device=device)#для оценки качества

num_epochs = 5
learning_rate = 1e-3

img_list = []
G_losses = []
D_losses = []
iters = 0

optimizerD = optim.Adam(D.parameters(), lr=learning_rate)
optimizerG = optim.Adam(G.parameters(), lr=learning_rate)

criterion = nn.BCELoss()
i=0
for epoch in range(num_epochs):
    for (data, _) in loader:
        i+=1

        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))

        # train with real
        D.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        
        label = torch.full((batch_size,), 1, device=device)

        output = D(real_cpu)
        D_loss1 = criterion(output, label)
        D_loss1.backward()
        D_x = output.mean().item()

        # train with fake
        z = torch.randn(1, latent_size, 1, 1)
        fake_photo = G(z)
        label.fill_(0)
        output = D(fake.detach())
        D_loss2 = criterion(output, label)
        D_loss2.backward()

        D_loss = D_loss1 + D_loss2
        optimizerD.step()

        # (2) Update G network: maximize log(D(G(z)))

        G.zero_grad()
        label.fill_(1)  # fake labels are real for generator cost
        output = D(fake_photo)
        G_loss = criterion(output, label)
        G_loss.backward()
        optimizerG.step()
        
        if iters % 10 == 0:
            # Выведем информацию о том, как наша сеть справляется
            print(f'{epoch}/{num_epochs}, {iters/len(loader)}')
            print(f'  G loss: {G_loss}')
            print(f'  D loss: {D_loss}')
            print()
            
        if i % 50 == 0:
            fake = G(fixed_noise)
            img_list.append(fake)



In [0]:
num_epochs = 5
learning_rate = 1e-3

img_list = []
G_losses = []
D_losses = []
iters = 0

optim_G = # ваш любимый оптимизатор параметров дискриминатора
optim_D = # ваш любимый оптимизатор параметров генератора

for epoch in range(num_epochs):
    for (data, _) in loader:
        # Обучать GANы всегда долго, и мы хотим по максимуму переиспользовать вычисления

        # 1. Обучим D: max log(D(x)) + log(1 - D(G(z)))
        
        D.zero_grad()
        
        # a) Распакуйте данные на нужный девайс
        #    Прогоните через сеть
        #    Сгенерируйте вектор из единичек (ответы для реальных сэмплов)
        #    Посчитайте лосс, сделайте .backward()
        # b) Посэмплите из torch.randn
        #    Прогоните этот шум через генератор
        #    detach-ните (нам не нужно считать градиенты G)
        #    Прогоните через дискриминатор
        #    Сгенерите вектор из нулей (ответы для фейков)
        #    Посчитайте лосс, сделайте backward (он сложится, а не перезапишется)
        #
        #    Также можно сначала сгенерировать данные, а потом собрать из двух частей батч,
        #    В котором первая половина лэйблов будет нулями, а вторая -- единицами
        
        optim_D.step()
        

        # 2. Обучим G: max log(D(G(z)))

        G.zero_grad()
        
        # Тут проще:
        #    Получим вектор неправильных ответов -- вектор единиц (мы ведь хотим, чтобы D считал их неправильными)
        #    Прогоним ранее сгенерированные картинки через D
        #    Посчитаем лосc, сделаем .backward()
        
        optim_G.step()

        # Раз в сколько-то итераций логгируем лосс
        if iters % 10 == 0:
            # Выведем информацию о том, как наша сеть справляется
            print(f'{epoch}/{num_epochs}, {iters/len(loader)}')
            print(f'  G loss: {G_loss}')
            print(f'  D loss: {D_loss}')
            print()
            
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())

        if iters % 50 == 0:
            # вы на этом батче уже генерировали какие-то картинки: просто добавьте их в список

        iters += 1

In [0]:
plt.figure(figsize=(10,5))
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [0]:
# распечатайте ваши картинки

### Что дальше?

Довольно старый, но актуальный список трюков: https://github.com/soumith/ganhacks

Вообще, теория сходимости GANов очень сильно развилась за последнее время. Если хотите во всём этом разобраться, то возьмите какую-нибудь [достаточно новую статью](https://arxiv.org/pdf/1802.05957.pdf) и рекурсивно почитайте оттуда абстракты из списока литературы.